In [1]:
import os
import numpy as np 
from PIL import Image
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Function to find Pixel values of Images
def get_pixel_values(image_path):
    img = Image.open(image_path)
    return list(img.getdata())


# Assigning Digits to Every Emotion

In [3]:
emotionList=["angry","disgusted","fearful","happy","neutral","sad","surprised"]
DigitAssigned={"angry":1,"disgusted":2,"fearful":3,"happy":4,"neutral":5,"sad":6,"surprised":7}

In [4]:
#Find out Number of Images (Not Necessary)
no_of_images=0
for emotion in emotionList:
    no_of_images+=len(os.listdir(f"/kaggle/input/emotion-detection-fer/train/{emotion}"))
no_of_images    

28709

# Training and Testing Data

In [5]:
# Training Data
all_image_pixel_values = []
all_emotion_labels = []

for emotion in emotionList:
    images = os.listdir(f"/kaggle/input/emotion-detection-fer/train/{emotion}")
    image_paths = [f"/kaggle/input/emotion-detection-fer/train/{emotion}/" + img for img in images]
    image_pixel_values = [get_pixel_values(img_path) for img_path in image_paths]
    all_image_pixel_values.extend(image_pixel_values)
    all_emotion_labels.extend([DigitAssigned[emotion]] * len(image_pixel_values))

X_train = np.array(all_image_pixel_values)
y_train = np.array(all_emotion_labels)

In [6]:
X_train.shape

(28709, 2304)

In [7]:
y_train.shape

(28709,)

In [8]:
#Randomizing the Training Distribution
indices = np.random.permutation(len(X_train))
X_train = X_train[indices]
y_train = y_train[indices]

In [9]:
X_train

array([[ 45,  61,  51, ..., 110,  51,  52],
       [103, 105,  85, ..., 240, 238, 238],
       [128, 126, 125, ...,  27,  29,  36],
       ...,
       [124, 128,  70, ..., 138, 137, 136],
       [215, 215, 215, ..., 166, 167, 169],
       [109, 110, 100, ...,  89,  88,  87]])

In [10]:
y_train

array([3, 4, 3, ..., 1, 5, 4])

In [11]:
# Test Data
all_image_pixel_values = []
all_emotion_labels = []

for emotion in emotionList:
    images = os.listdir(f"/kaggle/input/emotion-detection-fer/test/{emotion}")
    image_paths = [f"/kaggle/input/emotion-detection-fer/test/{emotion}/" + img for img in images]
    image_pixel_values = [get_pixel_values(img_path) for img_path in image_paths]
    all_image_pixel_values.extend(image_pixel_values)
    all_emotion_labels.extend([DigitAssigned[emotion]] * len(image_pixel_values))


X_test = np.array(all_image_pixel_values)
y_test = np.array(all_emotion_labels)

In [12]:
X_test.shape

(7178, 2304)

In [13]:
y_test.shape

(7178,)

In [14]:
#Dividing by 255 to normalize
X_train=X_train/255.0
X_test=X_test/255.0

In [15]:
#Reshaping the array
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [16]:
X_train.shape

(28709, 48, 48, 1)

In [17]:
#One hot encoding
y_train= keras.utils.to_categorical(y_train)[:,1:]
y_test = keras.utils.to_categorical(y_test)[:,1:]

In [18]:
#Creating Validation Set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [19]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
#Necessary imports
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2

# Model Creation

In [21]:

def Emotion_Model(input_shape=(48,48,1)):
    inputImg = Input(shape=input_shape, name='input')
    num_classes = 7

    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(inputImg)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)
    
    
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    conv2_3 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)
    
    
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)
    
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    
    
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
    conv5_3 = BatchNormalization()(conv5_4)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(7, activation='softmax', name = 'output')(flatten)
    model = Model(inputs =inputImg, outputs = ouput)
    
    print(model.summary())
    
    return model

In [23]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, 
                              restore_best_weights=True,mode="max",patience = 5),
                ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")]

In [26]:
#Model Compilation
model = Emotion_Model()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 64)        256       
 chNormalization)                                                
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0     

In [28]:
#Training the Model
ModelHistory = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=64,   
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=(X_val, y_val))

Epoch 1/100


2023-11-18 14:21:09.531056: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/drop1_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


449/449 [==============================] - ETA: 0s - loss: 2.1545 - accuracy: 0.2381
Epoch 1: val_accuracy improved from -inf to 0.25327, saving model to best_model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


449/449 [==============================] - 60s 85ms/step - loss: 2.1545 - accuracy: 0.2381 - val_loss: 1.8635 - val_accuracy: 0.2533
Epoch 2/100
449/449 [==============================] - ETA: 0s - loss: 1.6315 - accuracy: 0.3539
Epoch 2: val_accuracy improved from 0.25327 to 0.40791, saving model to best_model.h5
449/449 [==============================] - 38s 84ms/step - loss: 1.6315 - accuracy: 0.3539 - val_loss: 1.5247 - val_accuracy: 0.4079
Epoch 3/100
449/449 [==============================] - ETA: 0s - loss: 1.4156 - accuracy: 0.4545
Epoch 3: val_accuracy improved from 0.40791 to 0.46893, saving model to best_model.h5
449/449 [==============================] - 38s 85ms/step - loss: 1.4156 - accuracy: 0.4545 - val_loss: 1.3711 - val_accuracy: 0.4689
Epoch 4/100
449/449 [==============================] - ETA: 0s - loss: 1.2934 - accuracy: 0.5009
Epoch 4: val_accuracy improved from 0.46893 to 0.51491, saving model to best_model.h5
449/449 [==============================] - 39s 88ms/

In [29]:
#Calculating Loss
loss = model.evaluate(X_test,y_test)
print("Test Acc: " + str(loss[1]))

113/113 [==============================] - 2s 13ms/step - loss: 1.1005 - accuracy: 0.6562
Test Acc: 0.6561716198921204


In [30]:
#Saving the Model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
